In [1]:
!pip install transformers soundfile


In [9]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from google.colab import files
import time


In [15]:

def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer




def inference(file, model, processor, ref_text, noise="-"):

  speech_array, sampling_rate = torchaudio.load(file)
  speech_array = speech_array.squeeze()
  if sampling_rate != 16000:
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    speech_array = resampler(speech_array)
    sampling_rate = 16000

  speech = speech_array.numpy()
  inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

  with torch.no_grad():
      curr1 = time.time()
      logits = model(inputs.input_values).logits
      curr2 = time.time()
  pred_ids = torch.argmax(logits, dim=-1)
  text = processor.decode(pred_ids[0])
  wer = calculate_wer(ref_text, text)
  print("\n=== Výsledek rozpoznávání ===")
  print(f"Rozpoznaný text : {text}")
  print(f"CZ reference    : {ref_text}")

  print("\n=== Statistiky ===")
  print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
  print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise:<10}")


In [13]:
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print("File have been uploaded:", audio_path)


Saving jezero_snr0.ogg to jezero_snr0.ogg
Saving jezero_snr10.ogg to jezero_snr10.ogg
Saving jezero_snr20.ogg to jezero_snr20.ogg
Saving jezero_snr30.ogg to jezero_snr30.ogg
File have been uploaded: jezero_snr0.ogg


In [11]:
cz_reference = "Modrý stín pomalu přeplouvá přes tichou hladinu jezera."

processor = Wav2Vec2Processor.from_pretrained("arampacha/wav2vec2-large-xlsr-czech")
model = Wav2Vec2ForCTC.from_pretrained("arampacha/wav2vec2-large-xlsr-czech")


/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [16]:
inference("jezero.ogg", model, processor, cz_reference, noise="-")
inference("jezero_snr0.ogg", model, processor, cz_reference, noise="0")
inference("jezero_snr10.ogg", model, processor, cz_reference, noise="10")
inference("jezero_snr20.ogg", model, processor, cz_reference, noise="20")
inference("jezero_snr30.ogg", model, processor, cz_reference, noise="30")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r


=== Výsledek rozpoznávání ===
Rozpoznaný text : modrý jsti pomalu pře plouva přestichou hladinu gezera
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
8.173           0.750      -         

=== Výsledek rozpoznávání ===
Rozpoznaný text : nekdodiki omalu předloula coti flově hladini lěze
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
3.779           0.875      0         

=== Výsledek rozpoznávání ===
Rozpoznaný text : nadristým pomalu před dlouva přistichou hladin gesera
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
4.740           0.875      10        

=== Výsledek rozpoznávání ===
Rozpoznaný text : modrý sti pomalu pře plouva přestichou hladěnu gesera
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== St